### Introduction

The purpose of this model is to solve the famous transportation problem, that is also called the distribution problem. where the goal is to minimize the cost of the distribution of products from supply locations (origin) simulate, to the destinations where they are demand at minimum costs. In this model, each of sources has a limited amount of goods available each month, and each destinations has a monthly requirement for those goods. An array of costs exists; these costs to ship one unit of the single type of good from each of the possibly sources or origins to each of the potentially many sinks or destinations.

### Problem Statment 

Suppose two factories are manufacturing and supplying a single type of good to four warehouses in different parts of the country. The factories can produce a1 and a2 units of the good each month. and the warehouses need b1, b2, b3, and b4 units each month. The monthly total of goods available from the two factories exceeds the total of the monthly demands at the four warehouses. The costs of supplying one unit of the good from factory i to warehouse j is cii· the two factories have different factors of production. Their energy and labor costs are very different, with one of the factories having much higher electricity rates and employee wages. The costs of production of one unit of the good al factories 1 and 2 are d1 and d2. 
![alt text](diagram.png)

For a problem with two sources and four destinations, the formulation is:

$$Minimize\quad Z = (d_1 + c_{11})x_{11} + (d_1+c_{12})x_{12} + (d_1+c_{13})x_{13}+(d_1+c_{14})x_{14} +$$
$$ (d_2 + c_{21})x_{21} + (d_2+c_{22})x_{22} + (d_2+c_{23})x_{23} + (d_2+c_{24})x_{24}$$

Subject to:
$$x_{11} + x_{12} + x_{13} + x_{14} \leq a_1$$
$$x_{21} + x_{22} + x_{23} + x_{24} \leq a_2$$
$$x_{11} + x_{21} = b_1$$
$$x_{12} + x_{22} = b_2$$
$$x_{13} + x_{23} = b_3$$
$$x_{14} + x_{24} = b_4$$

and 
$$x_{ij} \geq 0$$

In [20]:
import geopandas
from libpysal import examples
import matplotlib
import mip
import numpy
import os
import spaghetti
import matplotlib_scalebar
from matplotlib_scalebar.scalebar import ScaleBar

%matplotlib inline
#%watermark -w
# %watermark -iv

In [21]:
supply_factories = [1, 6]
demand_warehouses = [2, 3, 4, 5]

In [22]:
amount_supply = [20, 30, 15, 35]
amount_demand = [5, 45, 10, 40]

In [33]:
class TransportationModel:
    def __init__(
        self,
        supply_nodes,
        demand_nodes,
        cij,
        si,
        dj,
        xij_tag="x_%s,%s",
        supply_constr_tag="supply(%s)",
        demand_constr_tag="demand(%s)",
        solver="cbc",
        display=True,
    ):
        """Instantiate and solve the Primal Transportation Problem
        based the formulation from Daskin (2013, Ch. 2).

        Parameters
        ----------
        supply_nodes : geopandas.GeoSeries
            Supply node decision variables.
        demand_nodes : geopandas.GeoSeries
            Demand node decision variables.
        cij : numpy.array
            Supply-to-demand distance matrix for nodes.
        si : geopandas.GeoSeries
            Amount that can be supplied by each supply node.
        dj : geopandas.GeoSeries
            Amount that can be received by each demand node.
        xij_tag : str
            Shipping decision variable names within the model. Default is
            'x_%s,%s' where %s indicates string formatting.
        supply_constr_tag : str
            Supply constraint labels. Default is 'supply(%s)'.
        demand_constr_tag : str
            Demand constraint labels. Default is 'demand(%s)'.
        solver : str
            Default is 'cbc' (coin-branch-cut). Can be set
            to 'gurobi' (if Gurobi is installed).
        display : bool
            Print out solution results.

        Attributes
        ----------
        supply_nodes : See description in above.
        demand_nodes : See description in above.
        cij : See description in above.
        si : See description in above.
        dj : See description in above.
        xij_tag : See description in above.
        supply_constr_tag : See description in above.
        demand_constr_tag : See description in above.
        rows : int
            The number of supply nodes.
        rrows : range
            The index of supply nodes.
        cols : int
            The number of demand nodes.
        rcols : range
            The index of demand nodes.
        model : mip.model.Model
            Integer Linear Programming problem instance.
        xij : numpy.array
            Shipping decision variables (``mip.entities.Var``).
        """

        # all nodes to be visited
        self.supply_nodes, self.demand_nodes = supply_nodes, demand_nodes
        
        # shipping costs (distance matrix) and amounts
        self.cij, self.si, self.dj = cij, si.values, dj.values
        self.ensure_float()
        
        # alpha tag for decision variables
        self.xij_tag = xij_tag
        
        # alpha tag for supply and demand constraints
        self.supply_constr_tag = supply_constr_tag
        self.demand_constr_tag = demand_constr_tag

        # instantiate a model
        self.model = mip.Model(" TransportationProblem", solver_name=solver)
        
        # define row and column indices
        self.rows, self.cols = self.si.shape[0], self.dj.shape[0]
        self.rrows, self.rcols = range(self.rows), range(self.cols)
        
        # create and set the decision variables
        self.shipping_dvs()
        
        # set the objective function
        self.objective_func()
        
        # add supply constraints
        self.add_supply_constrs()
        
        # add demand constraints
        self.add_demand_constrs()
        
        # solve
        self.solve(display=display)
        
        # shipping decisions lookup
        self.get_decisions(display=display)

    def ensure_float(self):
        """Convert integers to floats (rough edge in mip.LinExpr)"""
        self.cij = self.cij.astype(float)
        self.si = self.si.astype(float)
        self.dj = self.dj.astype(float)

    def shipping_dvs(self):
        """Create the shipping decision variables - eq (4)."""

        def _s(_x):
            """Helper for naming variables"""
            return self.supply_nodes[_x].split("_")[-1]

        def _d(_x):
            """Helper for naming variables"""
            return self.demand_nodes[_x].split("_")[-1]

        xij = numpy.array(
            [
                [self.model.add_var(self.xij_tag % (_s(i), _d(j))) for j in self.rcols]
                for i in self.rrows
            ]
        )
        self.xij = xij

    def objective_func(self):
        """Add the objective function - eq (1)."""
        self.model.objective = mip.minimize(
            mip.xsum(
                self.cij[i, j] * self.xij[i, j] for i in self.rrows for j in self.rcols
            )
        )

    def add_supply_constrs(self):
        """Add supply contraints to the model - eq (2)."""
        for i in self.rrows:
            rhs, label = self.si[i], self.supply_constr_tag % i
            self.model += mip.xsum(self.xij[i, j] for j in self.rcols) <= rhs, label

    def add_demand_constrs(self):
        """Add demand contraints to the model - eq (3)."""
        for j in self.rcols:
            rhs, label = self.dj[j], self.demand_constr_tag % j
            self.model += mip.xsum(self.xij[i, j] for i in self.rrows) >= rhs, label

    def solve(self, display=True):
        """Solve the model"""
        self.model.optimize()
        if display:
            obj = round(self.model.objective_value, 4)
            print("Minimized shipping costs: %s" % obj)

    def get_decisions(self, display=True):
        """Fetch the selected decision variables."""
        shipping_decisions = {}
        if display:
            print("\nShipping decisions:")
        for i in self.rrows:
            for j in self.rcols:
                v, vx = self.xij[i, j], self.xij[i, j].x
                if vx > 0:
                    if display:
                        print("\t", v, vx)
                    shipping_decisions[v.name] = vx
        self.shipping_decisions = shipping_decisions

    def print_lp(self, name=None):
        """Save LP file in order to read in and print."""
        if not name:
            name = self.model.name
        lp_file_name = "%s.lp" % name
        self.model.write(lp_file_name)
        lp_file = open(lp_file_name, "r")
        lp = lp_file.read()
        print("\n", lp)
        lp_file.close()
        os.remove(lp_file_name)

    def extract_shipments(self, paths, id_col, ship="ship"):
        """Extract the supply to demand shipments as a
        ``geopandas.GeoDataFrame`` of ``shapely.geometry.LineString`` objects.

        Parameters
        ----------
        paths : geopandas.GeoDataFrame
            Shortest-path routes between all ``self.supply_nodes``
            and ``self.demand_nodes``.
        id_col : str
            ID column name.
        ship : str
            Column name for the amount of good shipped.
            Default is 'ship'.

        Returns
        -------
        shipments : geopandas.GeoDataFrame
            Optimal shipments from ``self.supply_nodes`` to
            ``self.demand_nodes``.
        """

        def _id(sp):
            """ID label helper"""
            return tuple([int(i) for i in sp.split("_")[-1].split(",")])

        paths[ship] = int
        # set label of the shipping path for each OD pair.
        for ship_path, shipment in self.shipping_decisions.items():
            paths.loc[(paths[id_col] == _id(ship_path)), ship] = shipment
        # extract only shiiping paths
        shipments = paths[paths[ship] != int].copy()
        shipments[ship] = shipments[ship].astype(int)

        return shipments

In [ ]:
def solve(self, display=True):
        """Solve the model"""
        self.model.optimize()
        if display:
            obj = round(self.model.objective_value, 4)
            print("Minimized shipping costs: %s" % obj)

In [ ]:
def get_decisions(self, display=True):
        """Fetch the selected decision variables."""
        shipping_decisions = {}
        if display:
            print("\nShipping decisions:")
        for i in self.rrows:
            for j in self.rcols:
                v, vx = self.xij[i, j], self.xij[i, j].x
                if vx > 0:
                    if display:
                        print("\t", v, vx)
                    shipping_decisions[v.name] = vx
        self.shipping_decisions = shipping_decisions